In [7]:

import sys
import os
# Agregar el directorio '' al sys.path
sys.path.append(os.path.abspath('../'))


In [12]:
from Function import cargar_dataset
from  limpiar_customer_id import limpiar_customer_id
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from openpyxl import load_workbook 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [11]:
file_path = '/Users/Usuario/Desktop/prueba_citec/data_set/OnlineRetail.xlsx'
# Cargar el dataset
df = cargar_dataset(file_path)

In [14]:
df = limpiar_customer_id(df)

# Prepare the data

In [15]:
# Crear una columna con la fecha de la última compra
df['ultima_compra'] = df.groupby('CustomerID')['InvoiceDate'].transform('max')

In [16]:
# Calcular los días desde la última compra
df['dias_desde_ultima_compra'] = (df['InvoiceDate'].max() - df['ultima_compra']).dt.days

In [17]:
# Crear la variable objetivo: 1 si el cliente compró en los últimos 30 días, 0 si no
df['compra_30_dias'] = np.where(df['dias_desde_ultima_compra'] <= 30, 1, 0)

In [18]:
# Seleccionar las características y la variable objetivo
X = df[['dias_desde_ultima_compra']]
y = df['compra_30_dias']

In [19]:
# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Building the neural network

In [22]:
# Crear el modelo
modelo = Sequential()

# Añadir la capa de entrada y una capa oculta
modelo.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))

# Añadir la capa de salida
modelo.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\Usuario\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Train the model

In [24]:
# Evaluar el modelo
resultados = modelo.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {resultados[1]*100:.2f}%")

5081/5081 ━━━━━━━━━━━━━━━━━━━━ 4s 614us/step - accuracy: 0.7668 - loss: 0.6313
Precisión en el conjunto de prueba: 76.66%


#  Predict the probability of a new purchase

In [31]:
# Predecir la probabilidad de una nueva compra para un nuevo cliente
nuevos_datos = np.array([[1], [3000], [365]])
nuevos_datos_scaled = scaler.transform(nuevos_datos)
probabilidades = modelo.predict(nuevos_datos_scaled)
print("Probabilidades de nueva compra en los próximos 30 días:", probabilidades)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Probabilidades de nueva compra en los próximos 30 días: [[0.5534646 ]
 [0.67991126]
 [0.5212735 ]]


C:\Users\Usuario\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
